## 数据集切分

In [25]:
import pandas as pd
import numpy as np
train = pd.read_csv('data/nCoV_100k_train.labled.csv',header=0)
test = pd.read_csv('data/nCov_10k_test.csv',header=0)
train = train[(train['情感倾向'].isin(['-1','0','1']))]
train = train[~train.index.isin(['35202','41624','33936'])].reset_index(drop=True)

In [26]:
col = ['微博id','微博中文内容','微博发布时间','情感倾向']
train['情感倾向'] = train['情感倾向'].astype('int')+1
test['情感倾向'] = -1
train = train[col]
test = test[col]
train.columns = ['id','title','content','label']
test.columns = ['id','title','content','label']

In [27]:
train['title'] = train['title'].astype('str')
train['content'] = train['content'].astype('str')
test['title'] = test['title'].astype('str')
test['content'] = test['content'].astype('str')

In [28]:
train['label'].value_counts(1)

1    0.576689
2    0.254149
0    0.169162
Name: label, dtype: float64

In [29]:
from sklearn.model_selection import StratifiedKFold
train_df = train.copy()
test_df = test.copy()
train_df['title'] = train_df['title'].astype('str')
train_df['content'] = train_df['content'].astype('str')
X = np.array(train_df.index)
y = train_df.loc[:,'label'].to_numpy()

In [30]:
import os
def generate_data(random_state = 24, is_pse_label=True):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    i = 0
    for train_index, dev_index in skf.split(X, y):
        print(i, "TRAIN:", train_index, "TEST:", dev_index)
        DATA_DIR = "./data/data_StratifiedKFold_{}/data_origin_{}/".format(random_state,i)
        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)
        tmp_train_df = train_df.iloc[train_index]
        
        tmp_dev_df = train_df.iloc[dev_index]
        
        test_df.to_csv(DATA_DIR+"test.csv")
        if is_pse_label:
            pse_dir = "data_pse_{}/".format(i)
            pse_df = pd.read_csv(pse_dir+'train.csv')

            tmp_train_df = pd.concat([tmp_train_df, pse_df],ignore_index=True,sort=False)
            
        tmp_train_df.to_csv(DATA_DIR + "train.csv")
        tmp_dev_df.to_csv(DATA_DIR+"dev.csv")
        print(tmp_train_df.shape, tmp_dev_df.shape)
        i+=1

generate_data(random_state=48, is_pse_label=False)

0 TRAIN: [    0     1     2 ... 99906 99907 99908] TEST: [    3     5     6 ... 99893 99902 99909]
(79926, 4) (19984, 4)
1 TRAIN: [    0     1     2 ... 99907 99908 99909] TEST: [    4     7     8 ... 99880 99887 99900]
(79927, 4) (19983, 4)
2 TRAIN: [    0     2     3 ... 99906 99908 99909] TEST: [    1     9    11 ... 99890 99895 99907]
(79929, 4) (19981, 4)
3 TRAIN: [    0     1     3 ... 99907 99908 99909] TEST: [    2    16    18 ... 99897 99904 99905]
(79929, 4) (19981, 4)
4 TRAIN: [    1     2     3 ... 99905 99907 99909] TEST: [    0    10    12 ... 99903 99906 99908]
(79929, 4) (19981, 4)


In [ ]:
# chinese_roberta_wwm_ext_pytorch

In [8]:
!python ./bert_att__muldropout.py \
--model_type bert \
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_48/data_origin_0 \
--output_dir data/model_save/roberta_wwm_large_48_gru1_0 \
--max_seq_length 155 \
--split_num 1 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 32 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 32 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 3000 \
--freeze 0

04/06/2020 10:29:10 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/06/2020 10:29:10 - INFO - pytorch_transformers.tokenization_utils -   Model name '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' is a path or url to a directory containing tokenizer files.
04/06/2020 10:29:10 - INFO - pytorch_transformers.tokenization_utils -   Didn't find file /data/data01/liyang099/com/weigh

In [2]:
!python ./bert_last3embedding_cls.py \
--model_type bert \BertForSequenceClassification_muldropout
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_48/data_origin_1 \
--output_dir data/model_save/roberta_wwm_large_48_gru1_1 \
--max_seq_length 155 \
--split_num 1 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 64 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 64 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 2500 \
--freeze 0

04/02/2020 13:21:53 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/02/2020 13:21:53 - INFO - pytorch_transformers.tokenization_utils -   Model name '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' is a path or url to a directory containing tokenizer files.
04/02/2020 13:21:53 - INFO - pytorch_transformers.tokenization_utils -   Didn't find file /data/data01/liyang099/com/weigh

04/02/2020 13:37:52 - INFO - __main__ -     eval_F1 = 0.7238139960742004
04/02/2020 13:37:52 - INFO - __main__ -     eval_loss = 0.5539993637095625
04/02/2020 13:37:52 - INFO - __main__ -     global_step = 600
04/02/2020 13:37:52 - INFO - __main__ -     loss = 0.5674
Best F1 0.7238139960742004
Saving Model......
loss 0.5501: 100%|██████████████████████████| 2500/2500 [13:00<00:00,  3.20it/s]
04/02/2020 13:38:14 - INFO - pytorch_transformers.modeling_utils -   loading weights file data/model_save/roberta_wwm_large_48_gru1_1/pytorch_model.bin
--------- this is freeze: 0 ----------
dev 0.7238139960742004
/appcom/AnacondaInstall/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
test 0.0


In [29]:
!python ./bert_last3embedding_cls.py \
--model_type bert \
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_48/data_origin_2 \
--output_dir data/model_save/roberta_wwm_large_48_gru1_2 \
--max_seq_length 155 \
--split_num 1 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 64 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 64 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 2500 \
--freeze 0

04/02/2020 14:35:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/02/2020 14:35:58 - INFO - pytorch_transformers.tokenization_utils -   Model name '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' is a path or url to a directory containing tokenizer files.
04/02/2020 14:35:58 - INFO - pytorch_transformers.tokenization_utils -   Didn't find file /data/data01/liyang099/com/weigh

04/02/2020 14:51:55 - INFO - __main__ -     eval_F1 = 0.7246793740627414
04/02/2020 14:51:55 - INFO - __main__ -     eval_loss = 0.5471432026678
04/02/2020 14:51:55 - INFO - __main__ -     global_step = 600
04/02/2020 14:51:55 - INFO - __main__ -     loss = 0.5547
loss 0.5736: 100%|██████████████████████████| 2500/2500 [12:55<00:00,  3.22it/s]
04/02/2020 14:52:14 - INFO - pytorch_transformers.modeling_utils -   loading weights file data/model_save/roberta_wwm_large_48_gru1_2/pytorch_model.bin
--------- this is freeze: 0 ----------
dev 0.7273625201675661
/appcom/AnacondaInstall/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
test 0.0


In [3]:
!python ./bert_last3embedding_cls.py \
--model_type bert \
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_5566/data_origin_3 \
--output_dir data/model_save/roberta_wwm_large_48_gru1_3 \
--max_seq_length 155 \
--split_num 1 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 64 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 64 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 2500 \
--freeze 0

04/02/2020 13:41:08 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/02/2020 13:41:08 - INFO - pytorch_transformers.tokenization_utils -   Model name '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' is a path or url to a directory containing tokenizer files.
04/02/2020 13:41:08 - INFO - pytorch_transformers.tokenization_utils -   Didn't find file /data/data01/liyang099/com/weigh

04/02/2020 13:57:05 - INFO - __main__ -     eval_F1 = 0.7231968400529186
04/02/2020 13:57:05 - INFO - __main__ -     eval_loss = 0.5565289329416074
04/02/2020 13:57:05 - INFO - __main__ -     global_step = 600
04/02/2020 13:57:05 - INFO - __main__ -     loss = 0.5617
Best F1 0.7231968400529186
Saving Model......
loss 0.5482: 100%|██████████████████████████| 2500/2500 [13:00<00:00,  3.20it/s]
04/02/2020 13:57:27 - INFO - pytorch_transformers.modeling_utils -   loading weights file data/model_save/roberta_wwm_large_48_gru1_3/pytorch_model.bin
--------- this is freeze: 0 ----------
dev 0.7231968400529186
/appcom/AnacondaInstall/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
test 0.0


In [4]:
!python ./bert_last3embedding_cls.py \
--model_type bert \
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_48/data_origin_4 \
--output_dir data/model_save/roberta_wwm_large_48_gru1_4 \
--max_seq_length 155 \
--split_num 1 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 64 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 64 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 5000 \
--freeze 0

04/02/2020 14:00:23 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/02/2020 14:00:23 - INFO - pytorch_transformers.tokenization_utils -   Model name '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming '/data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/' is a path or url to a directory containing tokenizer files.
04/02/2020 14:00:23 - INFO - pytorch_transformers.tokenization_utils -   Didn't find file /data/data01/liyang099/com/weigh

loss 0.5552:  80%|████████████████████▊     | 3999/5000 [17:26<03:05,  5.40it/s]04/02/2020 14:21:08 - INFO - __main__ -   ***** Report result *****
04/02/2020 14:21:08 - INFO - __main__ -     global_step = 1000
04/02/2020 14:21:08 - INFO - __main__ -     train loss = 0.5552
04/02/2020 14:21:54 - INFO - __main__ -   ***** Running evaluation *****
04/02/2020 14:21:54 - INFO - __main__ -     Num examples = 19981
04/02/2020 14:21:54 - INFO - __main__ -     Batch size = 64
04/02/2020 14:22:56 - INFO - __main__ -     eval_F1 = 0.7264775756486821
04/02/2020 14:22:56 - INFO - __main__ -     eval_loss = 0.5479369870009133
04/02/2020 14:22:56 - INFO - __main__ -     global_step = 1000
04/02/2020 14:22:56 - INFO - __main__ -     loss = 0.5552
Best F1 0.7264775756486821
Saving Model......
loss 0.5393:  96%|████████████████████████▉ | 4799/5000 [21:40<00:37,  5.41it/s]04/02/2020 14:25:22 - INFO - __main__ -   ***** Report result *****
04/02/2020 14:25:22 - INFO - __main__ -     global_step = 1200
0

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

train = pd.read_csv('data/nCoV_100k_train.labled.csv', header=0)
test = pd.read_csv('data/nCov_10k_test.csv', header=0)
train = train[(train['情感倾向'].isin(['-1', '0', '1']))]
train = train[~train.index.isin(['35202', '41624', '33936'])].reset_index(drop=True)
test = pd.read_csv('data/nCov_10k_test.csv',header=0)

In [36]:
path = '/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_48_gru1_0/'
sub = pd.read_csv(path + 'sub.csv')

In [31]:
sub.columns
sub1_col = [ 'label_0', 'label_1', 'label_2']
test['y'] = np.argmax(sub[sub1_col].values, axis=1) -1

In [14]:
test_best = np.loadtxt('data/model_save2/test_bert_prob_0.7354800953858035.txt')
test['y2'] = np.argmax(test_best, axis=1) - 1

In [32]:
test['y'].value_counts(1)

 0    0.5667
 1    0.2645
-1    0.1688
Name: y, dtype: float64

In [33]:
test[test['y'] == test['y2']].shape[0]/test.shape[0]

0.9207

In [47]:
test[test['y'] != test['y2']][['微博中文内容','y','y2']]

,微博中文内容,y,y2
1,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,-1,0
13,恋家的宝贝//@1215theCloudsilent_云默:回家就好了,0,1
17,人生的第一次期末考试前，他发烧了。?,0,-1
26,事实证明人在发烧时表现出来的才是最真实的自己。?,1,0
27,其實在拍照的時候我有點小發燒我這真的是燃燒??生命、用愛??發電?,1,0
...,...,...,...
9965,打造“高效、畅顺、平安、智慧、温馨春运”！广宁公安全力以赴保障春运交通安全1月2日下午，广宁...,0,1
9982,【#火神山医院连通远程会诊系统#】今天，解放军总医院与武汉火神山医院远程会诊系统实现连通，远...,1,0
9988,找点乐子还得过下去//@老李家最有钱的仔叫李大亮:！//@宋英杰:以前没有火神山和雷神山？,1,0
9991,黑鹰坠落。谁在任时买的？那个党派？会不会成为选票拐点？?,0,-1


## 模型构建

In [48]:
# !python ./bert_pytorch_base.py \
# --model_type bert \
# --model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
# --do_train \
# --do_eval \
# --do_test \
# --data_dir /data/data01/liyang099/com/multi_setiment_reg/data/data_StratifiedKFold_5566/data_origin_0 \
# --output_dir data/model_save/roberta_wwm_large_2562_gru1_0 \
# --max_seq_length 150 \
# --split_num 2 \
# --lstm_hidden_size 512 \
# --lstm_layers 1 \
# --lstm_dropout 0.1 \
# --eval_steps 200 \
# --per_gpu_train_batch_size 64 \
# --gradient_accumulation_steps 4 \
# --warmup_steps 0 \
# --per_gpu_eval_batch_size 64 \
# --learning_rate 5e-5 \
# --adam_epsilon 1e-6 \
# --weight_decay 0 \
# --train_steps 9000 \
# --freeze 0

In [ ]:
!python ./bert_pytorch_base.py \
--model_type bert \
--model_name_or_path /data/data01/liyang099/com/weight/torch/chinese_roberta_wwm_ext_pytorch/  \
--do_train \
--do_eval \
--do_test \
--data_dir /data/data01/liyang099/com/setiment_regnition/data/data_StratifiedKFold_5566/data_origin_1 \
--output_dir data/model_save/roberta_wwm_large_2562_gru1_0 \
--max_seq_length 150 \
--split_num 2 \
--lstm_hidden_size 512 \
--lstm_layers 1 \
--lstm_dropout 0.1 \
--eval_steps 200 \
--per_gpu_train_batch_size 64 \
--gradient_accumulation_steps 4 \
--warmup_steps 0 \
--per_gpu_eval_batch_size 64 \
--learning_rate 5e-5 \
--adam_epsilon 1e-6 \
--weight_decay 0 \
--train_steps 9000 \
--freeze 0

In [31]:
import pandas as pd
test = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_0/sub.csv')
oof_test = test[['label_0','label_1','label_2']].values
test['y'] = np.argmax(oof_test, axis=1) -1

In [37]:
test1 = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_0/sub.csv')
test2 = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_1/sub.csv')
test3 = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_2/sub.csv')
test4 = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_3/sub.csv')
test5 = pd.read_csv('/data/data01/liyang099/com/multi_setiment_reg/data/model_save/roberta_wwm_large_2562_gru1_4/sub.csv')
oof_test1 = test1[['label_0','label_1','label_2']].values
oof_test2 = test2[['label_0','label_1','label_2']].values
oof_test3 = test3[['label_0','label_1','label_2']].values
oof_test4 = test4[['label_0','label_1','label_2']].values
oof_test5 = test5[['label_0','label_1','label_2']].values
oof_test = (oof_test1+ oof_test2 + oof_test3 + oof_test4 +oof_test5)/5
test['y'] = np.argmax(oof_test, axis=1) -1
test.rename(columns={'微博id':'id'}, inplace=True)

In [ ]:
oof_test = oof_test1+ oof_test2 + oof_test3 + oof_test4 +oof_test5
test['y'] = np.argmax(oof_test, axis=1) -1
test.rename(columns={'微博id':'id'}, inplace=True)

In [45]:
test.head(2)

,id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,y,y2
0,4456068992182160,01月01日 23:38,-精緻的豬豬女戰士-,#你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...,['https://ww2.sinaimg.cn/thumb150/745aa591ly1g...,[],0,0
1,4456424178427250,01月02日 23:09,liujunyi88,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,[],[],0,0


In [47]:
test[['id', 'y']].to_csv('/data/data01/liyang099/com/multi_setiment_reg/data/submit/submit.csv', index=False)

In [36]:
# test2 = pd.read_csv('/data/data01/liyang099/com/setiment_regnition/data/submit/bert_0.717071140841614.csv')

In [38]:
test['y'].value_counts(1)

 0    0.5924
 1    0.2492
-1    0.1584
Name: y, dtype: float64

In [26]:
y[:10]

array([1, 0, 2, 2, 2, 0, 0, 1, 0, 2], dtype=int64)

In [48]:
coefficients = [1.0,1.0,0.65]
oof_test2= oof_test * coefficients
test['y'] = np.argmax(oof_test2, axis=1) - 1


In [49]:
test['y'].value_counts(1)

 0    0.5602
 1    0.2801
-1    0.1597
Name: y, dtype: float64

In [24]:
# -*- coding: utf-8 -*-
"""
@Time ： 2020/3/15 22:59
@Auth ： joleo
@File ：optimized_rounder.py
"""
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    # https://www.kaggle.com/teejmahal20/regression-with-optimized-rounder
    """

    def __init__(self):
        self.coef_ = 0

    def _macro_f1_score(y_true, y_pred, n_labels):
        # https://www.kaggle.com/corochann/fast-macro-f1-computation
        total_f1 = 0.
        for i in range(n_labels):
            yt = y_true == i
            yp = y_pred == i

            tp = np.sum(yt & yp)

            tpfp = np.sum(yp)
            tpfn = np.sum(yt)
            if tpfp == 0:
                print('[WARNING] F-score is ill-defined and being set to 0.0 in labels with no predicted samples.')
                precision = 0.
            else:
                precision = tp / tpfp
            if tpfn == 0:
                print(f'[ERROR] label not found in y_true...')
                recall = 0.
            else:
                recall = tp / tpfn
            if precision == 0. or recall == 0.:
                f1 = 0.
            else:
                f1 = 2 * precision * recall / (precision + recall)
            total_f1 += f1
        return total_f1 / n_labels

    def _f1_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients

        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = coef * np.copy(X)

        return -f1_score(y, np.argmax(X_p, axis=-1), average='macro')

    def fit(self, X, y):
        """
        Optimize rounding thresholds

        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._f1_loss, X=X, y=y)
        initial_coef = [1.0,1.1,1.0]#[1.0 for _ in range(len(set(y)))]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, y):
        """
        Make predictions with specified thresholds

        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        X_p = self.coef_['x'] * np.copy(X)

        return f1_score(y, np.argmax(X_p, axis=-1), average='macro')

    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']